In [ ]:
import tensorflow_recommenders as tfrs
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_gnn as tfgnn
import numpy as np

In [ ]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
ratings = tfds.as_dataframe(ratings)
ratings = ratings.astype({"movie_id":int, "user_zip_code":object, "user_id":int}).sort_values("user_id")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")
users = ratings.drop_duplicates("user_id").sort_values("user_id")

In [ ]:
movies_ = ratings.drop_duplicates("movie_id").sort_values("movie_id")[["movie_genres", "movie_id", "movie_title"]]

In [ ]:
graph = tfgnn.GraphTensor.from_pieces(
    node_sets={
        "user": tfgnn.NodeSet.from_fields(
            sizes=[len(users)],
            features={
                "age":tf.convert_to_tensor(users["raw_user_age"].values),
                "occupation":tf.convert_to_tensor(users["user_occupation_label"].values),
                "gender":tf.convert_to_tensor(users["user_gender"].values),
                "user_id": tf.convert_to_tensor(users["user_id"].values)
            }
        ),

        "items": tfgnn.NodeSet.from_fields(
            sizes=[len(movies_)],
            features={
                "movie_genres":tf.ragged.constant(movies_["movie_genres"].values),
                "movie_id":tf.convert_to_tensor(movies_["movie_id"].values),
            }
        )
    },

    edge_sets={
        "rating": tfgnn.EdgeSet.from_fields(
            sizes=[100_000],
            adjacency=tfgnn.Adjacency.from_indices(
                source=("user", tf.convert_to_tensor(ratings["movie_id"])),
                target=("items", tf.convert_to_tensor(ratings["user_id"]))
            ),

            features={
                "rating": tf.convert_to_tensor(ratings["user_rating"]),
                "timestamp": tf.convert_to_tensor(ratings["timestamp"])
            }
        )
    }
)